## Sampling & Splitting Data

## Part A: 분석을 위한 환경 설정

### 모듈/패키지 로드

In [1]:
evaluation = True
evaluation_verbose = False

OUTPUT_BUCKET_FOLDER = "gs://cap-18/output/"
DATA_BUCKET_FOLDER = "gs://cap-18/data/"

In [2]:
from IPython.display import display

In [3]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.linalg import Vectors, SparseVector, VectorUDT

In [4]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all" # 한 셀(cell)에서의 코드 실행 결과가 다 보이도록 설정

In [5]:
import numpy as np
import scipy.sparse

In [6]:
import warnings 
warnings.filterwarnings('ignore') # warning 메시지 반환되지 않도록 설정

In [7]:
import math
import datetime
import time
import itertools

In [8]:
import pickle

In [9]:
import random
random.seed(42)

In [10]:
import pandas as pd
%matplotlib inline

## Part B: 변수별 결측치 확인

In [11]:
# 테이블 로드
train_valid_merged_df = spark.read.parquet("gs://cap-18/output/train_subset_final")

In [12]:
## cf) 판다스 데이터 프레임 구조 깨지지 않게 보기:
pd.set_option('display.max_columns', 100)

In [13]:
documents_categories_schema = StructType(
                    [StructField("document_id_cat", IntegerType(), True),
                    StructField("category_id", IntegerType(), True),                    
                    StructField("confidence_level_cat", FloatType(), True)]
                    )

documents_categories_df = spark.read.schema(documents_categories_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"documents_categories.csv") \
                .alias('documents_categories').cache()
    
documents_categories_grouped_df = documents_categories_df.groupBy('document_id_cat') \
                                            .agg(F.collect_list('category_id').alias('category_id_list'),
                                                 F.collect_list('confidence_level_cat').alias('confidence_level_cat_list')) \
                                            .withColumn('dummyDocumentsCategory', F.lit(1)) \
                                            .alias('documents_categories_grouped')

In [14]:
documents_topics_schema = StructType(
                    [StructField("document_id_top", IntegerType(), True),
                    StructField("topic_id", IntegerType(), True),                    
                    StructField("confidence_level_top", FloatType(), True)]
                    )

documents_topics_df = spark.read.schema(documents_topics_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"documents_topics.csv")  \
                .alias('documents_topics').cache()
    
documents_topics_grouped_df = documents_topics_df.groupBy('document_id_top') \
                                            .agg(F.collect_list('topic_id').alias('topic_id_list'),
                                                 F.collect_list('confidence_level_top').alias('confidence_level_top_list')) \
                                            .withColumn('dummyDocumentsTopics', F.lit(1)) \
                                            .alias('documents_topics_grouped')

다음의 코드로 변수별 결측치를 한번에 확인할 수 있다.

In [18]:
## cf) 판다스 100행까지 보기:
pd.set_option('display.max_rows', 100)

In [19]:
# train_valid_merged_df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in train_valid_merged_df.columns]).toPandas().transpose()

,0
display_id,0
ad_id,0
label,0
doc_id,0
is_leak,0
event_weekend,0
user_views,55311626
ad_views,33833186
doc_views,52700407
doc_event_hour,0


~~`user_views`, `ad_views`, `doc_views`는 참조할 사전 데이터가 없는 경우에 발생하기 때문에 1로 채워넣었다.~~

`user_views`, `ad_views`, `doc_views`는 일단 변수에서 빼자.

In [ ]:
train_imputing_df = train_valid_merged_df

train_imputing_df = train_imputing_df.drop('user_views', 'ad_views', 'doc_views')

In [27]:
# train_imputing_df = train_imputing_df.na.fill({'user_views': 1, 'ad_views': 1, 'doc_views': 1})

`traffic_source`와 `platform_event`는 최빈값인 1로 채우자.

In [ ]:
train_imputing_df = train_imputing_df.na.fill({'traffic_source': 1, 'platform_event': 1})

In [ ]:
train_imputing_df = train_imputing_df.withColumnRenamed('doc_event_source_id', 'view_source_id') \
                                    .withColumnRenamed('doc_event_publisher_id', 'view_publisher_id') \
                                    .drop('platform_event', 'pop_source_id_country_conf_multipl',
                                         'pop_source_id_country_conf', 'pop_source_id_country',
                                         'pop_category_id_country_conf_multipl', 'pop_category_id_country_conf',
                                         'pop_category_id_country')

pop-으로 시작하는 CTR과 그 외 범주형 변수의 결측치는 알려지지 않음(U: Unknown)으로 대체하자.

In [28]:
train_imputing_df = train_imputing_df.na.fill({'ad_publisher_id': 'U', 'ad_source_id': 'U', 'view_topic_id': 'U',
                                              'view_category_id': 'U', 'ad_topic_id': 'U', 'ad_category_id': 'U',
                                              'view_source_id': 'U', 'view_publisher_id': 'U',
                                              'view_publisher_id' : 'U', 'view_source_id' : 'U'})


#                                                'ad_category_conf': 'U'
#                                                'view_category_conf' : 'U', 'ad_topic_conf' : 'U'
#                                                'pop_category_id_conf_multipl' : 'U',
#                                                'pop_category_id_conf' : 'U',
#                                                'pop_category_id' : 'U',
#                                                'pop_topic_id_conf_multipl' : 'U',
#                                                'pop_topic_id_conf' : 'U',
#                                                'pop_topic_id' : 'U',
#                                                'pop_source_id_conf_multipl' : 'U',
#                                                'pop_source_id_conf' : 'U',
#                                                'pop_source_id' : 'U',
#                                                'pop_doc_event_doc_ad_conf_multipl' : 'U',
#                                                'pop_doc_event_doc_ad_conf' : 'U',
#                                                'pop_doc_event_doc_ad' : 'U',
#                                                'pop_campaign_id_conf_multipl' : 'U',
#                                                'pop_campaign_id_conf' : 'U',
#                                                'pop_campaign_id' : 'U',
#                                                'pop_advertiser_id_conf_multipl' : 'U',
#                                                'pop_advertiser_id_conf' : 'U',
#                                                'pop_advertiser_id' : 'U',
#                                                'pop_publisher_id_conf_multipl' : 'U',
#                                                'pop_publisher_id_conf' : 'U',
#                                                'pop_publisher_id' : 'U',
#                                                'pop_document_id_conf_multipl' : 'U',
#                                                'pop_document_id_conf' : 'U',
#                                                'pop_document_id' : 'U',
#                                                'pop_ad_id_conf_multipl' : 'U',
#                                                'pop_ad_id_conf' : 'U',
#                                                'pop_ad_id' : 'U'


# ad_publisher_id 유난히 NA가 많음 => 확인 필요 => okay. 원래 데이터에서도 그렇다.
# document_id_top, document_id_cat 삭제 => okay. Feature Enginnering_05 코드에 반영하였음. 재실행 시 해당 변수가 사라져 있을 것!

### cf) topic_id, category_id에 결측치가 있는 이유

 `view_topic_id`, `ad_topic_id`와 `view_category_id`, `ad_category_id`에 결측치는 왜 생긴 걸까? 
 
 원래 없는 정보였을지 파일을 합치는 과정에서 생긴 오류인지 체크해보자.

In [66]:
# # ad_topic_id가 Null인 경우
# null_ad_topic_id = train_valid_merged_df.where(F.col("ad_topic_id").isNull())

In [67]:
# null_ad_topic_id.select('ad_doc_id').distinct().count()

14550

In [64]:
# # ad_category_id가 Null인 경우
# null_ad_category_id = train_valid_merged_df.where(F.col("ad_category_id").isNull())

In [65]:
# null_ad_category_id.select('ad_doc_id').distinct().count()

533

In [68]:
# # view_topic_id가 Null인 경우
# null_view_topic_id = train_valid_merged_df.where(F.col("view_topic_id").isNull())

In [69]:
# null_view_topic_id.select('view_doc_id').distinct().count()

70911

In [70]:
# # view_category_id가 Null인 경우
# null_view_category_id = train_valid_merged_df.where(F.col("view_category_id").isNull())

In [71]:
# null_view_category_id.select('view_doc_id').distinct().count()

22170

`documents_categories` 테이블과 `documents_topics` 테이블을 확인한 결과, category_id와 topic_id가 원래 없었던 정보임을 확인할 수 있었다.

In [72]:
# l = documents_categories_grouped_df.select("document_id_cat").rdd.flatMap(lambda x: x).collect()

In [ ]:
# null_view_category_id.select('view_doc_id').distinct().filter(F.col('view_doc_id').isin(l)).show()

In [ ]:
# null_ad_category_id.select('ad_doc_id').distinct().filter(F.col('ad_doc_id').isin(l)).show()

In [ ]:
# l2 = documents_topics_grouped_df.select('document_id_top').rdd.flatMap(lambda x: x).collect()

In [ ]:
# null_view_topic_id.select('view_doc_id').distinct().filter(F.col('view_doc_id').isin(l2)).show()

In [ ]:
# null_ad_topic_id.select('view_doc_id').distinct().filter(F.col('view_doc_id').isin(l2)).show()

## Part C: OHE(One-Hot Encoding)

연속형 변수와 범주형 변수 모두 결국에는 범주형 변수로 전환할 예정이지만, 처리 과정이 상이하기 때문에 변수 형태별로 데이터를 구분해서 처리해보자.

In [ ]:
train_ohe_df = train_imputing_df

In [ ]:
# 범주형 변수: 32개

train_ohe_cat_df = train_ohe_df.drop('pop_ad_id', 'pop_ad_conf',
                                       'pop_ad_id_cont_multipl', 'pop_document_id',
                                       'pop_document_id_conf', 'pop_document_id_conf_multipl',
                                       'pop_advertiser_id', 'pop_advertiser_id_conf',
                                       'pop_advertiser_id_conf_multipl',
                                       'pop_campaign_id', 'pop_campaign_id_conf',
                                       'pop_campaign_id_conf_multipl', 
                                       'pop_doc_event_doc_ad', 'pop_doc_event_doc_ad_conf',
                                       'pop_doc_event_doc_ad_conf_multipl',
                                       'pop_source_id', 'pop_source_id_conf',
                                       'pop_source_id_conf_multipl',
                                       'pop_topic_id', 'pop_topic_id_conf',
                                       'pop_topic_id_conf_multipl',
                                       'pop_category_id', 'pop_category_id_conf',
                                       'pop_category_id_conf_multipl',
                                       'ad_topic_conf', 'view_topic_conf')

In [ ]:
# 범주형 변수의 수준(level) 수 확인

for col in train_ohe_cat_df.columns: # 범주형 변수인 column 만 subset
    print(col, train_ohe_cat_df.select(col).distinct().count()) # 고유 값을 가지는 행의 개수 세기

In [ ]:
# case 1) 범주형 변수의 수준 수가 특정 임계값을 넘지 않는 경우:

# 'other'

COUNT_THRESHOLD = 150 # 특정 임계값 설정 
 
# create a temporary col "count" as counting for each value of "prod_feat_3"
prodFeat3Count = df.groupBy("prod_feat_3").count()
df = df.join(prodFeat3Count, "prod_feat_3", "inner")
 
def convertMinority(originalCol, colCount):
    if colCount > COUNT_THRESHOLD:
        return originalCol
    else:
        return 'MinorityCategory'
    
createNewColFromTwo = udf(convertMinority, StringType())
df = df.withColumn('prod_feat_3_reduced', createNewColFromTwo(df['prod_feat_3'], df['count']))
df = df.drop('prod_feat_3')
df = df.drop('count')

In [ ]:
# case 2) 범주형 변수의 값이 NA인 경우:

# 'unknown'

In [ ]:
# case 3) 범주형 변수의 값이 없었던 값인 경우: 

# 'other' / 'unknown'

In [ ]:
# 연속형 변수: 32개

train_ohe_cont_df = train_ohe_df.select('pop_ad_id', 'pop_ad_conf',
                                       'pop_ad_id_cont_multipl', 'pop_document_id',
                                       'pop_document_id_conf', 'pop_document_id_conf_multipl',
                                       'pop_advertiser_id', 'pop_advertiser_id_conf',
                                       'pop_advertiser_id_conf_multipl',
                                       'pop_campaign_id', 'pop_campaign_id_conf',
                                       'pop_campaign_id_conf_multipl', 
                                       'pop_doc_event_doc_ad', 'pop_doc_event_doc_ad_conf',
                                       'pop_doc_event_doc_ad_conf_multipl',
                                       'pop_source_id', 'pop_source_id_conf',
                                       'pop_source_id_conf_multipl',
                                       'pop_topic_id', 'pop_topic_id_conf',
                                       'pop_topic_id_conf_multipl',
                                       'pop_category_id', 'pop_category_id_conf',
                                       'pop_category_id_conf_multipl',
                                       'ad_topic_conf', 'view_topic_conf')

In [ ]:
# 연속형 변수 binning ) 

# quantile 로 분할: 4개 + 1개(unknown) => 5개

# 1Q, 2Q, 3Q, 4Q 경계값을 계산하고, 이를 구준으로 4개로 분할

# data = [(1, 1.0),
#         (2, 2.0),
#         (3, 3.0),
#         (4, 4.0),
#         (5, 5.0),
#         (6, 6.0),
#         (7, 7.0),
#         (8, 8.0),
#         (9, 9.0),
#         (10, 10.0),
#         (11, None)]

# df = spark.createDataFrame(data, ["id", "hour"]) # data Frame으로 변환

# splits = [-float("inf"), 2.5, 5.0, 7.5, float("inf")]

# bucketizer = Bucketizer(splits=splits,
#                         inputCol="hour",
#                         outputCol="bucketed_hour",
#                         handleInvalid="keep")
# bucketedData = bucketizer.transform(df)
# bucketedData.show()
    
# df.approxQuantile(variable, [0.25, 0.5, 0.75], 0.0) # Q1, Q2, Q3 경계값 계산

# var_split = []

# for col in train_ohe_cont_df.columns: # 범주형 변수인 column 만 subset
#     print(col, train_ohe_cont_df.approxQuantile(col, [0.5], 0.0)) # 고유 값을 가지는 행의 개수 세기


# obj = {}

# for col in train_ohe_cont_df.columns:
#     obj[str(col)] = [train_ohe_cont_df.approxQuantile(col, [0.25, 0.5, 0.75], 0.0)]

In [ ]:
# # one-hot encoding categorical cols

# column_vec_in = ['prod_feat_3_reduced', 'cust_region', 'prod_type', 'cust_sex', 'cust_title']
# column_vec_out = ['prod_feat_3_reduced_catVec','cust_region_catVec', 'prod_type_catVec','cust_sex_catVec',
# 'cust_title_catVec']
 
# indexers = [StringIndexer(inputCol=x, outputCol=x+'_tmp')
#             for x in column_vec_in ]
 
# encoders = [OneHotEncoder(dropLast=False, inputCol=x+"_tmp", outputCol=y)
# for x,y in zip(column_vec_in, column_vec_out)]
# tmp = [[i,j] for i,j in zip(indexers, encoders)]
# tmp = [i for sublist in tmp for i in sublist]

In [ ]:
# # prepare labeled sets

# cols_now = ['prod_price',
#             'prod_feat_1',
#             'prod_feat_2',
#             'cust_age',
#             'prod_feat_3_reduced_catVec',
#             'cust_region_catVec',
#             'prod_type_catVec',
#             'cust_sex_catVec',
#             'cust_title_catVec']
# assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')
# labelIndexer = StringIndexer(inputCol='binary_response', outputCol="label")
# tmp += [assembler_features, labelIndexer]
# pipeline = Pipeline(stages=tmp)

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

`train`에서 세 개의 컬럼만 빼서 logistic regression으로 적합시켜보자.

In [ ]:
# train_light = train.select('label', 'event_weekend', 'doc_event_hour', 'pop_advertiser_id')

결측치가 포함되어 있으면 VectorAssembler가 작동하지 않는다.

In [ ]:
# train_light = train_light.dropna() ### 테스트 용으로 생성하였음. 

In [ ]:
categorical_columns= ['event_weekend', 'doc_event_hour']

indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
    for c in categorical_columns
]

encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

numericCols = ["pop_advertiser_id"]

assemblerInputs = [encoder.getOutputCol() for encoder in encoders] + numericCols
assembler = VectorAssembler(inputCols= assemblerInputs, outputCol="features")

In [ ]:
pipeline = Pipeline(stages=indexers + encoders+[assembler])
model=pipeline.fit(train_light)

In [ ]:
transformed = model.transform(train_light)
transformed.show(5)

In [ ]:
lrModel = LogisticRegression().fit(transformed)

In [ ]:
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

## Part D: doc_views, user_views, ad_views 다시 계산(이후 진행 예정)

`doc_views`, `user_views`, `ad_views`에 왜 결측치가 이렇게 많을까?

### Building doc_views counter

In [ ]:
# 1) train_imputing_df의 view_doc_id와 page_views의 document_id가 같은 것만 filtering
# 2) page_views의 timestamp가 train_imputing_df의 timestamp_event보다 작은 것만 filtering
# 3) page_views의 행의 수를 count


### Building user_views counter

In [ ]:
# 1) train_imputing_df의 uuid와 page_views의 uuid가 같은 것만 filtering
# 2) page_views의 timestamp가 train_imputing_df의 timestamp_event보다 작은 것만 filtering
# 3) page_views의 행의 수를 count



### Building ad_views counter

In [ ]:
# 1) train_imputing_df에서 ad_id가 같은 행만 filtering
# 2) 현재 행보다 timestamp가 작은 행만 filtering
# 3) train_imputing_df에서 남은 행의 수를 count

